In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pyvista as pv
from typing import Tuple, List, Union,Dict
from pprint import pprint

In [2]:
# 预设信息
# 单元自由度数
ELEM_DOFS_NUM=6
# 单元节点数
ELEM_NODES_NUM=3
# 节点自由度数
NODE_DOFS_NUM=2
# 置大数法的放大系数
SCALE=1e6
# 边界条件的自定义数据格式
bc_type = np.dtype([('set_name', 'U20'),('dof',int),('value',float)])

# 计算标题
title='平面三角形常应变单元'

# materials
E=2e10 # pa
nu=0.3

# displacements boundary conditions
is_displacement_boundary_condition=True
# 设置节点位移边界条件:定义集合bcx,bcy的相应自由度的边界条件值为0.0
bcs=np.array([('bcx',1,0.0),
              ('bcy',2,0.0)],dtype=bc_type)

# 节点力加载边界条件
is_concentrated_force=True
# 在集合load125上施加125N的力,方向为自由度2;同理,在集合load250上施加250N的力,方向为自由度2
direct_loads=np.array([('load125',2,125.0),
                       ('load250',2,250.0)],dtype=bc_type)

# 体力边界条件
is_body_force=False

# 面力边界条件
is_surface_force=False

# 输出选项
is_export_to_txt=True
result_file='PS3R_result.txt'

is_show_contour=True
is_deformed=True
is_show_stress=True

In [3]:
# 函数定义
def tri_area(p1:np.ndarray,p2:np.ndarray,p3:np.ndarray)->float:
    """根据三角形三个顶点坐标计算三角形面积"""
    x1,y1=p1[0],p1[1]
    x2,y2=p2[0],p2[1]
    x3,y3=p3[0],p3[1]
    return 0.5 * ((x2 - x1) * (y3 - y1) - (x3 - x1) * (y2 - y1))

def iso_2dStiff(E:float,nu:float)->np.ndarray:
    """计算平面刚度矩阵(3x3)"""
    return (E/(1-nu*nu))*np.array([[1.0,nu,0.0],
                     [nu,1.0,0.0],
                     [0.0,0.0,0.5*(1-nu)]])

def shap_func(p1:np.ndarray,p2:np.ndarray,p3:np.ndarray):
    """计算形函数,返回N(x,y)"""
    x1,y1=p1[0],p1[1]
    x2,y2=p2[0],p2[1]
    x3,y3=p3[0],p3[1]

    delta=np.linalg.det(np.array([[1,1,1],
                                  [x1,x2,x3],
                                  [y1,y2,y3]]))
    a1=np.linalg.det(np.array([[x2,y2],
                                [x3,y3]]))
    b1=-1.0*np.linalg.det(np.array([[1,y2],
                                [1,y3]]))
    c1=np.linalg.det(np.array([[1,x2],
                                [1,x3]]))

    a2=-1.0*np.linalg.det(np.array([[x1,y1],
                                    [x3,y3]]))
    b2=np.linalg.det(np.array([[1,y1],
                                [1,y3]]))
    c2=-1*np.linalg.det(np.array([[1,x1],
                                [1,x3]]))

    a3=np.linalg.det(np.array([[x1,y1],
                                [x2,y2]]))
    b3=-1.0*np.linalg.det(np.array([[1,y1],
                                [1,y2]]))
    c3=np.linalg.det(np.array([[1,x1],
                                [1,x2]]))

    def N(x:float,y:float)->np.ndarray:
        N1=0.5*(a1+b1*x+c1*y)/delta
        N2=0.5*(a2+b2*x+c2*y)/delta
        N3=0.5*(a3+b3*x+c3*y)/delta
        return np.array([[N1,0,N2,0,N3,0],
                         [0,N1,0,N2,0,N3]])

def Bmatrix(p1:np.ndarray,p2:np.ndarray,p3:np.ndarray)->np.ndarray:
    """计算B矩阵"""
    x1,y1=p1[0],p1[1]
    x2,y2=p2[0],p2[1]
    x3,y3=p3[0],p3[1]

    delta=tri_area(p1,p2,p3)

    b1=y2-y3
    c1=x3-x2
    b2=y3-y1
    c2=x1-x3
    b3=y1-y2
    c3=x2-x1

    N1x=0.5*b1/delta
    N1y=0.5*c1/delta
    N2x=0.5*b2/delta
    N2y=0.5*c2/delta
    N3x=0.5*b3/delta
    N3y=0.5*c3/delta

    return np.array([[N1x,0,N2x,0,N3x,0],
                     [0,N1y,0,N2y,0,N3y],
                     [N1y,N1x,N2y,N2x,N3y,N3x]])

def dof_index(node_label:int,dof:int)->int:
    """根据节点编号和自由度编号返回自由度索引,如节点10的2自由度对应dof_index(10,2)=11"""
    if dof==1:
        global_ind=2*node_label-1-1
    elif dof==2:
        global_ind=2*node_label-1
    else:
        pass
    return int(global_ind)

def pretty_matrix(arr:np.ndarray|list|tuple,returnStr:bool=False,format:int=0,prefix:str="")->None:
    """将数组,嵌套数组,列表等以格式化的形式输出到屏幕或者返回字符串,最多显示2维"""
    from collections.abc import Iterable
    def is_iterable(obj)->bool:
        return isinstance(obj, Iterable)

    formatStr=["%12.4e","%20.4e","%d","%.4f"]

    print(prefix)

    if returnStr:
        fstring=""

    # 不存在嵌套数组,就直接输出
    if not any(list(map(is_iterable,arr))):
        for i in arr:
            if returnStr:
                fstring+=formatStr[format]%i+"  "
            else:
                print(formatStr[format]%i,end="  ")
        if not returnStr :
            print()
        else:
            fstring+="\n"
    else:
        for i in arr:
            if is_iterable(i):
                for j in i:
                    if returnStr:
                        fstring+=formatStr[format]%j+"  "
                    else:
                        print(formatStr[format]%j,end="  ")
                if returnStr:
                    fstring+="\n"
                else:
                    print()
            else:
                if returnStr:
                    fstring+=formatStr[format]%i+"\n"
                else:
                    print(formatStr[format]%i)
    if returnStr:
        return fstring

In [4]:
# 读取网格信息
nodes=np.loadtxt('nodes.txt',delimiter=',')
elements=np.loadtxt('elems.txt',delimiter=',',dtype=int)

# 读取集合信息
with open('sets.txt','r') as f:
    nsets,names={},[]
    for line in f:
        if line.startswith('*set'):
            t1=line.strip().split(',')[1]
            names.append(t1.split('=')[1].strip())
            nsets[names[-1]]=[]
            continue
        datas=list(map(float,line.strip().split(',')))
        nsets[names[-1]]+=datas
for k,v in nsets.items():
    nsets[k]=np.array(v)


In [5]:
# 网格,节点,集合重编号

nodes_num = nodes.shape[0]
mesh_offset=-(nodes[:,0].min()-1)
elements[:,1:]=elements[:,1:]+mesh_offset
nodes[:,0]=np.arange(1,nodes_num+1)

for k,v in nsets.items():
    nsets[k]=v+mesh_offset

In [6]:
print(f"nodes=\n{nodes}\n elements=\n{elements}\nsets=\n{nsets}")

nodes=
[[ 1.   5.   5.   0. ]
 [ 2.   5.  10.   0. ]
 [ 3.   2.5  5.   0. ]
 [ 4.   0.   5.   0. ]
 [ 5.   0.   0.   0. ]
 [ 6.   2.5  0.   0. ]
 [ 7.   5.   0.   0. ]
 [ 8.   2.5 10.   0. ]
 [ 9.   0.  10.   0. ]]
 elements=
[[1 2 3 1]
 [2 5 3 4]
 [3 3 6 7]
 [4 3 8 9]
 [5 3 5 6]
 [6 7 1 3]
 [7 3 2 8]
 [8 9 4 3]]
sets=
{'bcx': array([5., 9., 4.]), 'bcy': array([5., 6., 7.]), 'load125': array([2., 9.]), 'load250': array([8.])}


In [7]:
# 构造全局刚度矩阵
nodes_num=nodes.shape[0]
K=np.zeros((NODE_DOFS_NUM*nodes_num,NODE_DOFS_NUM*nodes_num))

# 单元刚度矩阵
Ke=np.zeros((ELEM_DOFS_NUM,ELEM_DOFS_NUM))
from numpy import dot
# 迭代每个单元,计算单元刚度矩阵,叠加到全局刚度矩阵
for i in range(elements.shape[0]):
    connectivity=elements[i,1:]
    coord_tuple=(nodes[connectivity[0]-1,1:],nodes[connectivity[1]-1,1:],nodes[connectivity[2]-1,1:])
    # 单元面积
    S_tri=tri_area(*coord_tuple)
    # 单元B矩阵
    B=Bmatrix(*coord_tuple)
    # 计算应力-应变D矩阵
    D=iso_2dStiff(E,nu)
    # pretty_matrix(D,prefix='D')
    # 计算单元刚度矩阵Ke
    Ke=dot(B.T,dot(D,B))*S_tri
    # 单元的自由度列表(-1是因为python的索引从0开始)
    dof_inds=np.array([connectivity[0]*2-1,connectivity[0]*2,connectivity[1]*2-1,connectivity[1]*2,connectivity[2]*2-1,connectivity[2]*2])-1
    # 组装到全局刚度矩阵
    K[np.ix_(dof_inds,dof_inds)]+=Ke

pretty_matrix(K,prefix='global K')

global K
  4.7802e+10    0.0000e+00   -1.9231e+09    3.2967e+09   -4.3956e+10    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00   -1.9231e+09   -3.2967e+09    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00  
  0.0000e+00    2.6374e+10    3.8462e+09   -5.4945e+09    0.0000e+00   -1.5385e+10    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00   -3.8462e+09   -5.4945e+09    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00  
 -1.9231e+09    3.8462e+09    2.3901e+10    0.0000e+00    0.0000e+00   -7.1429e+09    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00   -2.1978e+10    3.2967e+09    0.0000e+00    0.0000e+00  
  3.2967e+09   -5.4945e+09    0.0000e+00    1.3187e+10   -7.1429e+09    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    3.8462e+09   -7.6923e+09    0.0000

In [8]:
# 处理力边界条件
# 全局等效节点力向量
F=np.zeros(NODE_DOFS_NUM*nodes_num)

if is_body_force:
    pass

if is_surface_force:
    pass

if is_concentrated_force:
    num_direct_load = direct_loads.shape[0]  # 直接荷载个数
    for i in range(num_direct_load):
        setName,dof,val=direct_loads[i]  # 荷载作用节点集
        for j in nsets[setName].tolist():  # 遍历荷载作用节点集
            F[dof_index(j,dof)]+=val  # 确定对应DOF的索引

pretty_matrix(F,prefix='设置加载后的全局等效节点力向量:')

设置加载后的全局等效节点力向量:
  0.0000e+00    0.0000e+00    0.0000e+00    1.2500e+02    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    2.5000e+02    0.0000e+00    1.2500e+02  


In [9]:
# 处理位移边界条件(置大数法)
# 全局节点位移向量
dofs_constraint=[]

# 复制变量,K_,F_用于后续计算
K_=K.copy()
F_=F.copy()

if is_displacement_boundary_condition:
    # 边界条件位移
    num_displacement_bc=bcs.shape[0]
    # 设置一个足够大的数
    C=abs(np.max(K))*SCALE
    for i in range(num_displacement_bc):
        set_name,dof,value=bcs[i]
        for j in nsets[set_name]:
            F_[dof_index(j,dof)]+=value*C
            K_[dof_index(j,dof),dof_index(j,dof)]+=C
            # 记录约束自由度的索引
            dofs_constraint.append(dof_index(j,dof))

pretty_matrix(F_,prefix="置大数法处理后的全局载荷向量为:")

pretty_matrix(K_,prefix="置大数法处理后的全局刚度矩阵为:")

置大数法处理后的全局载荷向量为:
  0.0000e+00    0.0000e+00    0.0000e+00    1.2500e+02    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    2.5000e+02    0.0000e+00    1.2500e+02  
置大数法处理后的全局刚度矩阵为:
  4.7802e+10    0.0000e+00   -1.9231e+09    3.2967e+09   -4.3956e+10    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00   -1.9231e+09   -3.2967e+09    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00  
  0.0000e+00    2.6374e+10    3.8462e+09   -5.4945e+09    0.0000e+00   -1.5385e+10    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00   -3.8462e+09   -5.4945e+09    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00  
 -1.9231e+09    3.8462e+09    2.3901e+10    0.0000e+00    0.0000e+00   -7.1429e+09    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00    0.0000e+00   -2.1978e+1

In [10]:
# 求解有限元方程: KU=F
# 计算节点位移
U=np.linalg.solve(K_, F_)
# 计算约束节点的支反力
# R=K*U
F_react=K.dot(U)

# 计算单元应变,应力
Elem_strains,Elem_stresses=np.zeros((elements.shape[0],4)),np.zeros((elements.shape[0],4))
for i in range(elements.shape[0]):
    connectivity=elements[i,1:]
    coord_tuple=(nodes[connectivity[0]-1,1:],nodes[connectivity[1]-1,1:],nodes[connectivity[2]-1,1:])
    # 计算B矩阵,D矩阵,Ke矩阵
    B=Bmatrix(*coord_tuple)
    D=iso_2dStiff(E,nu)
    # 单元自由度索引
    dof_inds=np.array([connectivity[0]*2-1,connectivity[0]*2,connectivity[1]*2-1,connectivity[1]*2,connectivity[2]*2-1,connectivity[2]*2])-1

    Elem_strains[i,0],Elem_stresses[i,0]=elements[i,0],elements[i,0]
    Elem_strains[i,1:]=dot(B,U[dof_inds]).reshape(-1)
    Elem_stresses[i,1:]=dot(D,Elem_strains[i,1:])



In [11]:
# 输出结果到文件
if is_export_to_txt:
    with open(result_file, 'w') as f:
        f.write(f"title = {title}\n\n")

        # 输出材料信息
        f.write(f"材料信息:\n")
        f.write(f"E = {E} , nu = {nu}\n\n")

        # 输出结果信息
        f.write('='*15+'nodal displacements'+'='*15+'\n')
        f.write(f"nodes label   U1 U2 \n")
        u=np.concatenate((nodes[:,0].reshape(-1,1),U[0::2].reshape(-1,1),U[1::2].reshape(-1,1)),axis=1)
        f.write(pretty_matrix(u,returnStr=True)+'\n\n')

        f.write('='*15+'nodal forces'+'='*15+'\n')
        f.write(f"nodes label   RF1 RF2 \n")
        rf=np.concatenate((nodes[:,0].reshape(-1,1),F_react[0::2].reshape(-1,1),F_react[1::2].reshape(-1,1)),axis=1)
        f.write(pretty_matrix(rf,returnStr=True)+'\n\n')

        f.write('='*15+'element stresses'+'='*15+'\n')
        f.write(f"element label   Sxx Syy Sxy \n")
        f.write(pretty_matrix(Elem_stresses,returnStr=True)+'\n\n')

        f.write('='*15+'element strains'+'='*15+'\n')
        f.write(f"element label   Exx Eyy Exy \n")
        f.write(pretty_matrix(Elem_strains,returnStr=True)+'\n\n')

In [12]:
# 后处理

pretty_matrix(U,prefix='U result=')

pretty_matrix(F_react,prefix='F result=')

U result=
 -7.5000e-09    2.5000e-08   -7.5000e-09    5.0000e-08   -3.7500e-09    2.5000e-08    8.6347e-24    2.5000e-08   -5.4451e-24    1.3075e-15   -3.7500e-09    2.6149e-15   -7.5000e-09    1.3075e-15   -3.7500e-09    5.0000e-08   -3.1897e-24    5.0000e-08  
F result=
  2.8422e-14    1.1369e-13    4.2633e-14    1.2500e+02    0.0000e+00   -1.1369e-13   -8.2552e-07    1.1369e-13    5.2057e-07   -1.2500e+02   -2.8422e-14   -2.5000e+02   -4.2633e-14   -1.2500e+02    0.0000e+00    2.5000e+02    3.0495e-07    1.2500e+02  


In [13]:
# pyvista show
from pyvista import CellType
pnodes=nodes[:,1:]
cells=np.concatenate((np.full((elements.shape[0], 1), elements[:,1:].shape[1]), elements[:,1:]-1), axis=1)
cells=cells.astype(int)
celltype=np.array([CellType.TRIANGLE] * cells.shape[0], dtype=int)

mesh = pv.UnstructuredGrid(cells, celltype, pnodes)

mesh.point_data['displacement']=np.concatenate((U[0::2].reshape(-1,1),U[1::2].reshape(-1,1)), axis=1)
mesh.cell_data['stress']=Elem_stresses[:,1:]
mesh.cell_data['strain']=Elem_strains[:,1:]


In [14]:
dargs = dict(
    scalars="displacement",
    cmap="jet",
    show_scalar_bar=True,
    show_edges=True,
)
pl = pv.Plotter(shape=(1, 3))
pl.subplot(0, 0)
pl.add_mesh(mesh, **dargs)
pl.add_text("Maginutde", color='k')
pl.subplot(0, 1)
pl.add_mesh(mesh.copy(), component=0, **dargs)
pl.add_text("U1", color='k')
pl.subplot(0, 2)
pl.add_mesh(mesh.copy(), component=1, **dargs)
pl.add_text("U2", color='k')
pl.link_views()
pl.camera_position = 'iso'
pl.background_color = 'white'
pl.show()

Widget(value='<iframe src="http://localhost:50913/index.html?ui=P_0x25c920f2d90_0&reconnect=auto" class="pyvis…

In [15]:

# show stress
dargs = dict(
    scalars="stress",
    cmap="jet",
    show_scalar_bar=True,
    show_edges=True,
)
pl = pv.Plotter(shape=(2, 2))
pl.subplot(0, 0)
pl.add_mesh(mesh, **dargs)
pl.add_text("Maginutde", color='k')
pl.subplot(0, 1)
pl.add_mesh(mesh.copy(), component=0, **dargs)
pl.add_text("Sxx", color='k')
pl.subplot(1, 0)
pl.add_mesh(mesh.copy(), component=1, **dargs)
pl.add_text("Syy", color='k')
pl.subplot(1, 1)
pl.add_mesh(mesh.copy(), component=2, **dargs)
pl.add_text("Sxy", color='k')
pl.link_views()
pl.camera_position = 'iso'
pl.background_color = 'white'
pl.show()

Widget(value='<iframe src="http://localhost:50913/index.html?ui=P_0x25c8f4bfcd0_1&reconnect=auto" class="pyvis…

In [16]:
# show streain
dargs = dict(
    scalars="strain",
    cmap="jet",
    show_scalar_bar=True,
    show_edges=True,
)
pl = pv.Plotter(shape=(2, 2))
pl.subplot(0, 0)
pl.add_mesh(mesh, **dargs)
pl.add_text("Maginutde", color='k')
pl.subplot(0, 1)
pl.add_mesh(mesh.copy(), component=0, **dargs)
pl.add_text("Exx", color='k')
pl.subplot(1, 0)
pl.add_mesh(mesh.copy(), component=1, **dargs)
pl.add_text("Eyy", color='k')
pl.subplot(1, 1)
pl.add_mesh(mesh.copy(), component=2, **dargs)
pl.add_text("Exy", color='k')
pl.link_views()
pl.camera_position = 'iso'
pl.background_color = 'white'
# pl.show()
pl.export_html("PS3R.html")